## Project 2: Expert

### A question answering agent that is an expert knowledge worker
### To be used by employees of Insurellm, an Insurance Tech company
### The agent needs to be accurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

We will be using the LangChain framework which does most of the heavy lifting for us! We'll also be using Gradio's chat interface, and Plotly for charts. We'll also be using the popular open source Vector Database Chroma.

## BEFORE WE START: take a look in the knowledge-base folder

In [1]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [17]:
# imports for langchain, plotly and Chroma

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

ModuleNotFoundError: No module named 'matplotlib'

In [3]:
# price is a factor for our company, so we're going to use a low cost model

MODEL = "gpt-4.1-mini"
db_name = "vector_db"

In [4]:
# Load environment variables in a file called .env

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


In [7]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

folders = glob.glob("knowledge-base/terraform_modules/*")

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs={'encoding': 'utf-8'})
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

print(f"Loaded {len(documents)} documents")

Loaded 154 documents


In [10]:
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

print(f"Divided into {len(chunks)} chunks")

Created a chunk of size 2290, which is longer than the specified 500
Created a chunk of size 659, which is longer than the specified 500
Created a chunk of size 3600, which is longer than the specified 500
Created a chunk of size 10252, which is longer than the specified 500
Created a chunk of size 560, which is longer than the specified 500
Created a chunk of size 5012, which is longer than the specified 500
Created a chunk of size 568, which is longer than the specified 500
Created a chunk of size 558, which is longer than the specified 500
Created a chunk of size 677, which is longer than the specified 500
Created a chunk of size 641, which is longer than the specified 500
Created a chunk of size 640, which is longer than the specified 500
Created a chunk of size 1340, which is longer than the specified 500
Created a chunk of size 15048, which is longer than the specified 500
Created a chunk of size 865, which is longer than the specified 500
Created a chunk of size 6076, which is l

Divided into 1765 chunks


## A sidenote on Embeddings, and "Auto-Encoding LLMs"

We will be mapping each chunk of text into a Vector that represents the meaning of the text, known as an embedding.

OpenAI offers a model to do this, which we will use by calling their API with some LangChain code.

This model is an example of an "Auto-Encoding LLM" which generates an output given a complete input.
It's different to all the other LLMs we've discussed today, which are known as "Auto-Regressive LLMs", and generate future tokens based only on past context.

Another example of an Auto-Encoding LLMs is BERT from Google. In addition to embedding, Auto-encoding LLMs are often used for classification.

More details in the resources.

In [11]:
# Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk
# Chroma is a popular open source Vector Database based on SQLLite

embeddings = OpenAIEmbeddings()

# Delete if already exists

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Create vectorstore

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 1765 documents


In [12]:
# Let's investigate the vectors

collection = vectorstore._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"There are {count:,} vectors with {dimensions:,} dimensions in the vector store")

There are 1,765 vectors with 1,536 dimensions in the vector store


## Visualizing the Vector Store

Let's take a minute to look at the documents and their embedding vectors to see what's going on.

In [14]:
# Prework

result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
metadatas = result['metadatas']
print(metadatas)
doc_types = [metadata.get('doc_type', 'unknown') for metadata in metadatas]
colors = [['blue', 'green', 'red', 'orange'][['azurerm_api_management', 'azurerm_data_factory', 'azurerm_databricks', 'azurerm_eventhub'].index(t)] for t in doc_types]



[{'doc_type': 'azurerm_api_management', 'source': 'knowledge-base\\terraform_modules\\azurerm_api_management\\README.md'}, {'doc_type': 'azurerm_api_management', 'source': 'knowledge-base\\terraform_modules\\azurerm_api_management\\README.md'}, {'doc_type': 'azurerm_api_management', 'source': 'knowledge-base\\terraform_modules\\azurerm_api_management\\README.md'}, {'doc_type': 'azurerm_api_management', 'source': 'knowledge-base\\terraform_modules\\azurerm_api_management\\README.md'}, {'source': 'knowledge-base\\terraform_modules\\azurerm_api_management\\README.md', 'doc_type': 'azurerm_api_management'}, {'doc_type': 'azurerm_api_management', 'source': 'knowledge-base\\terraform_modules\\azurerm_api_management\\README.md'}, {'doc_type': 'azurerm_api_management', 'source': 'knowledge-base\\terraform_modules\\azurerm_api_management\\README.md'}, {'source': 'knowledge-base\\terraform_modules\\azurerm_api_management\\README.md', 'doc_type': 'azurerm_api_management'}, {'source': 'knowledge-b

ValueError: 'azurerm_application_gateway' is not in list

In [18]:
from sklearn.manifold import TSNE
import plotly.graph_objects as go
import numpy as np

# 2. Pull embeddings + metadata from your existing vectorstore
collection = vectorstore._collection
result = collection.get(include=["embeddings", "metadatas"])
vectors = np.array(result["embeddings"])
metadatas = result["metadatas"]

# 3. Build hover text and doc_type lists
doc_types = [m["doc_type"] for m in metadatas]
hover_text = [
    f"{m['doc_type']}: {chunks[i].page_content[:100].replace(chr(10),' ')}…"
    for i, m in enumerate(metadatas)
]

# 4. Run t-SNE down to 2D
tsne = TSNE(n_components=2, perplexity=30, init="pca", random_state=42)
coords = tsne.fit_transform(vectors)

# 5. Assign one color per module
unique = list(dict.fromkeys(doc_types))
color_map = {t: f"hsl({i*360/len(unique):.0f},60%,50%)" for i, t in enumerate(unique)}

# 6. Plot with Plotly
fig = go.Figure()
for t in unique:
    idxs = [i for i, dt in enumerate(doc_types) if dt == t]
    fig.add_trace(go.Scatter(
        x=coords[idxs,0],
        y=coords[idxs,1],
        mode="markers",
        marker=dict(color=color_map[t], size=8),
        name=t,
        hovertext=[hover_text[i] for i in idxs],
        hoverinfo="text"
    ))
fig.update_layout(
    title="t-SNE of Terraform Module Chunks",
    xaxis_title="TSNE-Dim 1",
    yaxis_title="TSNE-Dim 2",
    legend_title="Module",
    width=800, height=600
)
fig.show()

In [19]:
# We humans find it easier to visalize things in 2D!
# Reduce the dimensionality of the vectors to 2D using t-SNE
# (t-distributed stochastic neighbor embedding)

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

NameError: name 'colors' is not defined

In [ ]:
# Let's try 3D!

tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

## Time to use LangChain to bring it all together

In [20]:
# create a new Chat with OpenAI
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# how many chunks to provide in each prompt
retriever = vectorstore.as_retriever(search_kwargs={"k": 35})

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

C:\Users\georg\AppData\Local\Temp\ipykernel_21772\751425390.py:5: LangChainDeprecationWarning:

Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/



In [21]:
# Let's try a simple question

query = "Please explain what Axso Terraform Self Service is?"
result = conversation_chain.invoke({"question": query})
print(result["answer"])

Axso Terraform Self Service is a set of Terraform modules designed to enable users to deploy cloud infrastructure in a self-service manner. These modules are intended to be used both interactively and within CI/CD pipelines, allowing teams to manage infrastructure efficiently and consistently.

Key points about Axso Terraform Self Service:

- It provides production-ready Terraform modules, services, and blueprints that have been rigorously tested, including compliance with Azure Policies defined by the Security Team.
- Users need access to the Axso Terraform Self Service project to utilize the modules and access valuable documentation such as How-To Guides.
- The modules support automation and are designed to integrate with Azure DevOps pipelines.
- The service follows a "Do Your Own System" (DYOS) approach, empowering teams to manage their own infrastructure deployments using standardized, compliant modules.

In summary, Axso Terraform Self Service is a platform and repository of test

## Now we will bring this up in Gradio using the Chat interface -

A quick and easy way to prototype a chat with an LLM

In [22]:
# Wrapping that in a function

def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [23]:
# Start the conversation again

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [24]:
# And launch Gradio

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


In [29]:
# Let's investigate what gets sent behind the scenes

from langchain_core.callbacks import StdOutCallbackHandler

llm = ChatOpenAI(temperature=0.7, model_name=MODEL)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    retriever=vectorstore.as_retriever(), 
    memory=memory,
    callbacks=[StdOutCallbackHandler()]
)

query = "How many terraform modules do we have? Count the .md files"
result = conversation_chain.invoke({"question": query})
answer = result["answer"]
print("\nAnswer:", answer)



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
In summary, this stage installs Terraform-Docs, updates README.md files for each Terraform module in the repository, and commits the updated files back to the repository.

# Terraform Files

----------------------------

## module.tf

1. [How-to-setup-your-terraform-pipeline](/Axso-Terraform-Modules/HOW-TO/How-to-setup-your-terraform-pipeline.md)
1. [Module-Versioning-Lifecycle](/Axso-Terraform-Modules/HOW-TO/Module-Versioning-Lifecycle.md)
1. [Understanding-Implicit-and-Explicit-Dependencies-in-Terraform](/Axso-Terraform-Modules/HOW-TO/Understanding-Implicit-and-Explicit-Dependencies-in-Terraform.md)
1. [Understanding-Count-and-ForEach-Loops-in-

# Major Assignment: Applying this to your business

Try applying this to your own folder of data, so that you create a personal knowledge worker, an expert on your own information!

## Stretch challenge

LangChain is very useful here, but it's not needed at all. There are benefits to implementing RAG manually, without relying on LangChain's prompts.

Give this a try - make an alternative implementation that bypasses LangChain!

## Stretch challenge #2

Implement hierarchical RAG, and demonstrate it answering a question correctly that it couldn't answer before